# 항공 지연 데이터 예측

In [1]:
install.packages("dplyr")
library(dplyr)

package 'dplyr' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\slyki\AppData\Local\Temp\RtmpOyj9mr\downloaded_packages


Warning message:
"package 'dplyr' was built under R version 3.6.1"
Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union



In [2]:
# 필요 데이터 로드

tryCatch(
    setwd("C:/workspace/R"),
    error = function(e) {
        setwd("D:/workspace/R")
    }
)

afsnt <- read.csv("Data/AFSNT.csv", header=T, stringsAsFactors = F)
afsnt_dly <- read.csv("Data/AFSNT_DLY.csv", header=T, stringsAsFactors = F)
sfsnt <- read.csv("Data/sfsnt.csv", header=T, stringsAsFactors = F)

## 1. 데이터 탐색
- 빅콘테스트 측에 제공한 항공데이터는 크게 3가지이다.
<br>

### 1) 운항실적 데이터
- 해당 데이터는 각 항공편 별 운항한 기록이 저장되어있으며, 운항일자, 출발지와 목적지, 항공사와 항공편명, 지연여부와 이유, 결항여부와 이유에 대한 자료가 같이 저장된다.
- 가장 먼저 확인해볼 자료는 저장된 기간을 확인해보자

In [3]:
str(afsnt)

'data.frame':	987709 obs. of  17 variables:
 $ SDT_YY: int  2017 2017 2017 2017 2017 2017 2017 2017 2017 2017 ...
 $ SDT_MM: int  1 1 1 1 1 1 1 1 1 1 ...
 $ SDT_DD: int  1 1 1 1 1 1 1 1 1 1 ...
 $ SDT_DY: chr  "일" "일" "일" "일" ...
 $ ARP   : chr  "ARP3" "ARP3" "ARP3" "ARP3" ...
 $ ODP   : chr  "ARP6" "ARP6" "ARP6" "ARP6" ...
 $ FLO   : chr  "J" "J" "J" "J" ...
 $ FLT   : chr  "J1955" "J1954" "J1956" "J1957" ...
 $ REG   : chr  "SEw3NzE4" "SEw3NzE4" "SEw3NzE4" "SEw3NzE4" ...
 $ AOD   : chr  "D" "A" "A" "D" ...
 $ IRR   : chr  "N" "N" "N" "N" ...
 $ STT   : chr  "10:05" "9:30" "12:45" "13:25" ...
 $ ATT   : chr  "10:32" "9:31" "13:03" "14:09" ...
 $ DLY   : chr  "N" "N" "N" "Y" ...
 $ DRR   : chr  "" "" "" "C02" ...
 $ CNL   : chr  "N" "N" "N" "N" ...
 $ CNR   : chr  "" "" "" "" ...


In [4]:
head(unique(afsnt[,1:4])); tail(unique(afsnt[,1:4]))

,SDT_YY,SDT_MM,SDT_DD,SDT_DY
,<int>,<int>,<int>,<chr>
1,2017,1,1,일
306,2017,1,2,월
604,2017,1,3,화
896,2017,1,4,수
1192,2017,1,5,목
1492,2017,1,6,금


,SDT_YY,SDT_MM,SDT_DD,SDT_DY
,<int>,<int>,<int>,<chr>
845740,2019,6,25,화
846042,2019,6,26,수
846344,2019,6,27,목
846643,2019,6,28,금
846961,2019,6,29,토
847270,2019,6,30,일


In [5]:
afsnt$AOD <- factor(afsnt$AOD)

In [6]:
afsnt$IRR <- factor(afsnt$IRR)

In [7]:
afsnt$DLY <- factor(afsnt$DLY)

In [8]:
afsnt$DRR <- factor(afsnt$DRR)

In [9]:
afsnt$CNL <- factor(afsnt$CNL)

In [10]:
afsnt$CNR <- factor(afsnt$CNR)

In [11]:
summary(afsnt)

     SDT_YY         SDT_MM           SDT_DD         SDT_DY         
 Min.   :2017   Min.   : 1.000   Min.   : 1.00   Length:987709     
 1st Qu.:2017   1st Qu.: 3.000   1st Qu.: 8.00   Class :character  
 Median :2018   Median : 6.000   Median :16.00   Mode  :character  
 Mean   :2018   Mean   : 5.956   Mean   :15.71                     
 3rd Qu.:2018   3rd Qu.: 9.000   3rd Qu.:23.00                     
 Max.   :2019   Max.   :12.000   Max.   :31.00                     
                                                                   
     ARP                ODP                FLO                FLT           
 Length:987709      Length:987709      Length:987709      Length:987709     
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                       

- 확인 결과 2017.01.01 ~ 2019.06.30의 운행정보가 저장되어있고, 지연여부를 목표변수라고 봤을 때, 위의 데이터는 학습용 데이터로 사용하는 것이 좋을 것으로 보인다.

### 2) 운항시즌 데이터 
- 해당 데이터는 2019년 7월~9월이 포함된 시즌 데이터이다. 
- 데이터에는 시즌에 대한 정보와 항공편명, 출발지와 도착지, 각 요일별 운행 여부,<br> 
  시작 일자와 종료 일자의 데이터를 담고 있다.

In [12]:
str(sfsnt)

'data.frame':	1186 obs. of  17 variables:
 $ SSC: chr  "S19" "S19" "S19" "S19" ...
 $ FLT: chr  "A1915" "A1904" "I1304" "B1802" ...
 $ ORG: chr  "ARP1" "ARP3" "ARP3" "ARP2" ...
 $ DES: chr  "ARP3" "ARP1" "ARP1" "ARP1" ...
 $ STD: chr  "7:55" "7:55" "8:00" "8:00" ...
 $ STA: chr  "9:05" "9:05" "9:15" "8:55" ...
 $ FLO: chr  "A" "A" "I" "B" ...
 $ MON: chr  "Y" "Y" "Y" "Y" ...
 $ TUE: chr  "Y" "Y" "" "Y" ...
 $ WED: chr  "Y" "Y" "Y" "Y" ...
 $ THU: chr  "Y" "Y" "Y" "Y" ...
 $ FRI: chr  "Y" "Y" "Y" "Y" ...
 $ SAT: chr  "Y" "Y" "" "Y" ...
 $ SUN: chr  "Y" "Y" "Y" "Y" ...
 $ FSD: int  20190331 20190331 20190331 20190331 20190730 20190331 20190902 20190827 20190401 20190729 ...
 $ FED: int  20191026 20191026 20191026 20191026 20190824 20190728 20191017 20191020 20190722 20190829 ...
 $ IRR: chr  "N" "N" "N" "N" ...


In [13]:
unique(sfsnt$SSC)

[1] "S19"

In [14]:
sfsnt$MON <- factor(sfsnt$MON);
sfsnt$TUE <- factor(sfsnt$TUE);
sfsnt$WED <- factor(sfsnt$WED);
sfsnt$THU <- factor(sfsnt$THU);
sfsnt$FRI <- factor(sfsnt$FRI);
sfsnt$SAT <- factor(sfsnt$SAT);
sfsnt$SUN <- factor(sfsnt$SUN);
sfsnt$IRR <- factor(sfsnt$IRR)

In [15]:
summary(sfsnt)

     SSC                FLT                ORG                DES           
 Length:1186        Length:1186        Length:1186        Length:1186       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                                                                            
     STD                STA                FLO            MON     TUE    
 Length:1186        Length:1186        Length:1186         :432    :436  
 Class :character   Class :character   Class :character   Y:754   Y:750  
 Mode  :character   Mode  :character   Mode  :character                  
                                                                         
                                                                         
                 

In [22]:
sort(unique(sfsnt$FSD))

[1] 20190331 20190401 20190402 20190403 20190404 20190405 20190406 20190407
  [9] 20190411 20190415 20190416 20190417 20190418 20190419 20190425 20190427
 [17] 20190428 20190429 20190430 20190501 20190503 20190504 20190505 20190507
 [25] 20190509 20190511 20190513 20190514 20190515 20190517 20190518 20190519
 [33] 20190520 20190521 20190522 20190523 20190524 20190525 20190526 20190527
 [41] 20190529 20190531 20190601 20190603 20190604 20190605 20190610 20190615
 [49] 20190617 20190618 20190619 20190624 20190625 20190626 20190627 20190630
 [57] 20190701 20190702 20190704 20190705 20190714 20190715 20190716 20190718
 [65] 20190723 20190724 20190725 20190729 20190730 20190731 20190801 20190802
 [73] 20190803 20190804 20190805 20190806 20190807 20190808 20190809 20190810
 [81] 20190812 20190813 20190826 20190827 20190828 20190830 20190831 20190901
 [89] 20190902 20190903 20190904 20190905 20190906 20190907 20190909 20190910
 [97] 20190911 20190914 20190916 20190917 20190922 20190923 20190924

- 시작일자와 종료일자가 있으며, 이는 이력 데이터와 유사하다고 보여진다.
- 단, 2019년 3월 ~ 9월까지 운항 정보이기 때문에 운항실적 데이터로 학습한 모델을 검증하는 용도로 사용할 수 있다.
- 따라서 앞선 데이터의 보충자료라고 판단된다.

### 3) 운항지연 데이터

- 운항실적 데이터와 형태는 유사하지만 데이터 정의서에 명시되어 있듯이, 운항일자는 2019년 9월 16 ~ 30일이며, 지연여부(DLY) 와 지연율(DLY_RATE) 변수가 NA 값으로 채워진 것으로 보아 해당데이터는 실제 예측해야 되는 테스트 셋으로 보여진다.

In [17]:
str(afsnt_dly)

'data.frame':	16076 obs. of  12 variables:
 $ SDT_YY  : int  2019 2019 2019 2019 2019 2019 2019 2019 2019 2019 ...
 $ SDT_MM  : int  9 9 9 9 9 9 9 9 9 9 ...
 $ SDT_DD  : int  16 16 16 16 16 16 16 16 16 16 ...
 $ SDT_DY  : chr  "월" "월" "월" "월" ...
 $ ARP     : chr  "ARP1" "ARP3" "`" "ARP3" ...
 $ ODP     : chr  "ARP3" "ARP1" "ARP3" "ARP1" ...
 $ FLO     : chr  "L" "L" "L" "L" ...
 $ FLT     : chr  "L1702" "L1702" "L1720" "L1720" ...
 $ AOD     : chr  "A" "D" "A" "D" ...
 $ STT     : chr  "9:05" "7:55" "14:40" "13:30" ...
 $ DLY     : logi  NA NA NA NA NA NA ...
 $ DLY_RATE: logi  NA NA NA NA NA NA ...


In [18]:
unique(paste0(afsnt_dly$SDT_YY, afsnt_dly$SDT_MM, afsnt_dly$SDT_DD))

[1] "2019916" "2019917" "2019918" "2019919" "2019920" "2019921" "2019922"
 [8] "2019923" "2019924" "2019925" "2019926" "2019927" "2019928" "2019929"
[15] "2019930"

**현재까지 정리된 내용**
- 크게 3가지 데이터 셋을 쓴다.
- 지연 여부와 그에 대한 확률을 고르는 문제이나 엄연히 분류에 대한 문제이다.(이진분류문제)
- afsnt > sfsnt > afsnt_dly 순으로 데이터를 접근하는 것이 좋다.

**TODO**
* 홈페이지에 공지된 내용 중 항공운항기록데이터 뿐만 아니라 항공기상데이터도 같이 사용하도록 공지되어있다. 따라서 학습 시 각 날짜별로 항공기상데이터도 같이 사용할 수 있도록 데이터 전처리를 한다.
* 날짜와 관련된 변수는 sfsnt 에 나온 것처럼 더미 변수(값의 여부를 위해 이진값으로 표시한 변수)를 생성해 주는 것이 좋다. 
* 분류 문제이기 때문에 분류 알고리즘을 사용하며 모델은 의사결정나무(CART), 렌덤 포레스트를 통해 사용할 만한 특징(Feature) 들을 찾는다.
* 앙상블, 신경망 등 Feature Selection에 사용된 기법외의 다른 기법들로 최종적인 지연 여부와 확률을 산출한다.
* 진행 시 사용한 모델이 적합한 지 검증도 같이 하며, 10-Fold 기법으로 검증한다. 
* 가능하다면 Kaggle 포커룰에 사용한 모델도 사용해볼 것

## 2. 데이터 전처리

**참고자료**
- R jupyter notebook 에 커널 추가 :  https://3months.tistory.com/332